# Setup

In [1]:
from download_artifact import download
import pandas as pd

## 1) Download run from mlflow

In [4]:
#run_id = "e01b366910bb4d86a94c9a6ee60cda66"
#download(run_id)

## 2) Use in Model Predictor

In [2]:
from Model import Model

In [3]:
test_model = Model()

03.10.2021 21:15:01 (INFO) load model
03.10.2021 21:15:13 (INFO) loading model done


In [4]:
test_text = ["Dans le couloir du métro, à Paris, en novembre 2015, François Duchâteau tombe face à une affiche de la COP21",
             "Comment ca va?",
             "Avez-vous le temps maintenant?",
             "C'est quelle date ?"]

In [5]:
test_model.predict(test_text)

03.10.2021 21:15:17 (INFO) received X ['Dans le couloir du métro, à Paris, en novembre 2015, François Duchâteau tombe face à une affiche de la COP21', 'Comment ca va?', 'Avez-vous le temps maintenant?', "C'est quelle date ?"]
03.10.2021 21:15:17 (INFO) start inference on X
03.10.2021 21:15:17 (INFO) executing argv ['perl', '/Users/christian/opt/anaconda3/envs/translators_mlflow/lib/python3.7/site-packages/mosestokenizer/split-sentences.perl', '-q', '-b', '-l', 'en', '-m']
03.10.2021 21:15:17 (WARNING) stdbuf was not found; communication with perl may hang due to stdio buffering.
03.10.2021 21:15:17 (INFO) spawned process 50694
03.10.2021 21:15:17 (DEBUG) << Dans le couloir du métro, à Paris, en novembre 2015, François Duchâteau tombe face à une affiche de la COP21
03.10.2021 21:15:17 (DEBUG) << <P>
03.10.2021 21:15:17 (DEBUG) readline()
03.10.2021 21:15:17 (DEBUG) >> Dans le couloir du métro, à Paris, en novembre 2015, François Duchâteau tombe face à une affiche de la COP21
03.10.2021 

raw model input                                                    0
0  Dans le couloir du métro, à Paris, en novembre...
1                                     Comment ca va?
2                     Avez-vous le temps maintenant?
3                                C'est quelle date ?
texts ['Dans le couloir du métro, à Paris, en novembre 2015, François Duchâteau tombe face à une affiche de la COP21', 'Comment ca va?', 'Avez-vous le temps maintenant?', "C'est quelle date ?"]
<class 'list'>


03.10.2021 21:15:17 (INFO) killing process 50695


_translate ['Dans le couloir du métro, à Paris, en novembre 2015, François Duchâteau tombe face à une affiche de la COP21', 'Comment ca va?', 'Avez-vous le temps maintenant?', "C'est quelle date ?"]


03.10.2021 21:15:20 (INFO) inference time [s]: 4
03.10.2021 21:15:20 (INFO) model output ['In the subway corridor in Paris in November 2015, François Duchâteau falls in front of a COP21 poster', 'How are you?', 'Do you have time now?', 'What date is it?']


['In the subway corridor in Paris in November 2015, François Duchâteau falls in front of a COP21 poster',
 'How are you?',
 'Do you have time now?',
 'What date is it?']

### STart as Microservice local
Debug flag enables using flask-internal werkzeug, not gunicorn,

In [ ]:
!seldon-core-microservice Model --service-type MODEL #--debug

In [6]:
! curl -s -X POST http://192.168.0.47:9000/api/v1.0/predictions \
    -H 'Content-Type: application/json'     -d '{ "data": { "ndarray": ["Aujourd hui est un bon jour"] } }'

^C


In [ ]:
# expect to return simply string "pong"
! curl http://192.168.0.47:9000/health/ping

In [ ]:
# do the model validation with simple example
! curl http://192.168.0.47:9000/health/status

## 3) Build Image

`make build`

### Test Locally

In [9]:
VERSION=!cat version.txt
VERSION=VERSION[0]

In [17]:
container_id = !docker run --rm -d -p 9000:9000/tcp translator:$VERSION
container_id = container_id[0]
container_id

'928eb55cda8438cf07ee50bb42c289a408b69b31c3cdb04bd0611986808e7e13'

In [18]:
! docker logs --tail 1000 $container_id

2021-10-04 11:18:05,991 - seldon_core.microservice:main:206 - INFO:  Starting microservice.py:main
2021-10-04 11:18:05,992 - seldon_core.microservice:main:207 - INFO:  Seldon Core version: 1.11.0
2021-10-04 11:18:05,995 - seldon_core.microservice:main:362 - INFO:  Parse JAEGER_EXTRA_TAGS []
2021-10-04 11:18:05,995 - seldon_core.microservice:main:365 - INFO:  Annotations: {}
2021-10-04 11:18:05,995 - seldon_core.microservice:main:369 - INFO:  Importing Model
2021-10-04 11:18:07,041 - seldon_core.microservice:main:448 - INFO:  REST gunicorn microservice running on port 9000
2021-10-04 11:18:07,044 - seldon_core.microservice:main:542 - INFO:  REST metrics microservice running on port 6000
2021-10-04 11:18:07,044 - seldon_core.microservice:main:552 - INFO:  Starting servers
2021-10-04 11:18:07,055 - seldon_core.microservice:grpc_prediction_server:505 - INFO:  GRPC Server Binding to '%s' 0.0.0.0:5000 with 1 processes
2021-10-04 11:18:07,074 - seldon_core.wrapper:_set_flask_app_configs:224 -

In [19]:
! curl -s -X POST http:///192.168.0.47:9000/api/v1.0/predictions \
    -H 'Content-Type: application/json' \
    -d '{ "data": { "ndarray": ["Aujourd hui est un bon jour"] } }'

{"data":{"names":[],"ndarray":["Today is a good day"]},"meta":{"tags":{"hostname":"928eb55cda84","image_version":"0.5","mode":"dev","model_version":"e01b366910bb4d86a94c9a6ee60cda66"}}}


In [20]:
! curl http://192.168.0.47:9000/health/status

{"data":{"names":[],"ndarray":["Today is a good day"]},"meta":{"tags":{"hostname":"928eb55cda84","image_version":"0.5","mode":"dev","model_version":"e01b366910bb4d86a94c9a6ee60cda66"}}}


### Push to Registry

In [25]:
! curl -s -X POST http://localhost:9001/api/v1.0/predictions \
    -H 'Content-Type: application/json' \
    -d '{ "data": { "ndarray": ["Aujourd hui est un bon jour"] } }'